<a href="https://colab.research.google.com/github/davide-ario/BNN/blob/main/BNN_gender_gap_solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from warnings import filterwarnings
import h5py
#h5py.run_tests()

import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import seaborn as sns
import sklearn
import theano
import theano.tensor as T
import tensorflow as tf

from sklearn import datasets
from sklearn.datasets import make_moons
#from scipy.special import comb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

In [ ]:
print(f"Running on PyMC3 v{pm.__version__}")


In [ ]:
%config InlineBackend.figure_format = 'retina'
floatX = theano.config.floatX
filterwarnings("ignore")
sns.set_style("white")

In [ ]:
# Input data files are available in the "../input/" directory.
import os
print("Input files:")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded["wage_data_reduced.csv"]),sep=";")
data= data.loc[:, ~data.columns.isin(['fit'])]
# Dataset is now stored in a Pandas Dataframe
data.info()
print()
print(data.shape)
print(data.columns)
data.head(5)

In [ ]:
Y = data.lnw
X= data.loc[:, data.columns != "lnw"]

In [ ]:
X = X.astype(floatX)
Y = Y.astype(floatX)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1)

In [ ]:
X.shape


In [ ]:

def construct_nnNEW(ann_input, ann_output):
    n_hidden = 10

    # Initialize random weights between each layer
    init_1 = np.random.randn(X.shape[1], n_hidden).astype(floatX)
    init_2 = np.random.randn(n_hidden, n_hidden).astype(floatX)
    init_out = np.random.randn(n_hidden).astype(floatX)
    init_b_1 = np.random.randn(n_hidden).astype(floatX)
    init_b_2 = np.random.randn(n_hidden).astype(floatX)
    init_b_3 = np.random.randn(n_hidden).astype(floatX)

    init_b_out = np.random.randn(1).astype(floatX)

#    init_1s = np.random.randn(X.shape[1], n_hidden).astype(floatX)
#    init_2s = np.random.randn(n_hidden, n_hidden).astype(floatX)
#    init_outs = np.random.randn(n_hidden).astype(floatX)
#    init_b_1s = np.random.randn(n_hidden).astype(floatX)
#    init_b_2s = np.random.randn(n_hidden).astype(floatX)
#    init_b_3s = np.random.randn(n_hidden).astype(floatX)
#
#    init_b_outs = np.random.randn(1).astype(floatX)






    with pm.Model() as neural_network:
        # Trick: Turn inputs and outputs into shared variables using the data container pm.Data
        # It's still the same thing, but we can later change the values of the shared variable
        # (to switch in the test-data later) and pymc3 will just use the new data.
        # Kind-of like a pointer we can redirect.
        # For more info, see: http://deeplearning.net/software/theano/library/compile/shared.html
        ann_input = pm.Data("ann_input", X_train)
        ann_output = pm.Data("ann_output", Y_train)
        #here I can set shapes of err and errs equalt to (1)
        err = pm.Normal("err", 0, sigma=1, shape=(X_train.shape[0],))
        errs = pm.Normal("errs", 0, sigma=1, shape=(X_train.shape[0],))

        model_sel=pm.Uniform('model_sel', lower=0, upper=1, shape=(X_train.shape[1],))
#        delta = pm.HalfCauchy('delta', 1)
        # Weights from input to hidden layer
        weights_in_1 = pm.Normal("w_in_1", 0, sigma=1, shape=(X.shape[1], n_hidden), testval=init_1)
        bias1 = pm.Normal("bias1", 0, sigma=1, shape=n_hidden, testval=init_b_1)

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal("w_1_2", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        bias2 = pm.Normal("bias2", 0, sigma=1, shape=n_hidden, testval=init_b_2)

#              # Weights from 1st to 2nd layer
#        weights_1_2 = pm.Normal("w_2_2", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        # Weights from hidden layer to output
        weights_2_2 = pm.Normal("w_2_2", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        bias3 = pm.Normal("bias3", 0, sigma=1, shape=n_hidden, testval=init_b_3)

        weights_2_out = pm.Normal("w_2_out", 0, sigma=1, shape=(n_hidden,), testval=init_out)

        # Build neural-network using tanh activation function
#        mu_para= alpha + ann_input[:,0]*beta[0]+ann_input[:,1]*beta[1]
        input_continuos = ann_input*(1-model_sel)
        act_1 = T.nnet.relu(pm.math.dot(input_continuos, weights_in_1)+bias1)
        act_2 = T.nnet.relu(pm.math.dot(act_1, weights_1_2)+bias2)
        act_3 = T.nnet.relu(pm.math.dot(act_2, weights_2_2)+bias3)
#        act_1 = T.dot(ann_input, weights_in_1)+err
#        act_2 = T.dot(act_1, weights_1_2)+err
#
#        act_3 = T.dot(act_2, weights_2_2)+err

#        # Build neural-network using tanh activation function

#        act_1 = T.dot(ann_input, weights_in_1)
#        act_2 = T.dot(act_1, weights_1_2)

        act_out = T.dot(act_3, weights_2_out)+err


#        errs = pm.Normal("errs", 0, sigma=1, shape=(X_train.shape[0],))

#        delta = pm.HalfCauchy('delta', 1)
        # Weights from input to hidden layer
        ##data to point
        
        ##
        weights_in_1s = pm.Normal("w_in_1s", 0, sigma=1, shape=(X.shape[1], n_hidden), testval=init_1)
        bias1s = pm.Normal("bias1s", 0, sigma=1, shape=n_hidden, testval=init_b_1)

        # Weights from 1st to 2nd layer
        weights_1_2s = pm.Normal("w_1_2s", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        bias2s = pm.Normal("bias2s", 0, sigma=1, shape=n_hidden, testval=init_b_2)

#              # Weights from 1st to 2nd layer
#        weights_1_2 = pm.Normal("w_2_2", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        # Weights from hidden layer to output
        weights_2_2s = pm.Normal("w_2_2s", 0, sigma=1, shape=(n_hidden, n_hidden), testval=init_2)
        bias3s = pm.Normal("bias3s", 0, sigma=1, shape=n_hidden, testval=init_b_3)

        weights_2_outs = pm.Normal("w_2_outs", 0, sigma=1, shape=(n_hidden,), testval=init_out)

        # Build neural-network using tanh activation function
#        mu_para= alpha + ann_input[:,0]*beta[0]+ann_input[:,1]*beta[1]
        input_discrete = ann_input*model_sel

        act_1s = pm.math.tanh(pm.math.dot(input_discrete, weights_in_1s)+bias1s)
        act_2s = pm.math.tanh(pm.math.dot(act_1s, weights_1_2s)+bias2s)
        act_3s = pm.math.tanh(pm.math.dot(act_2s, weights_2_2s)+bias3s)
#        act_1 = T.dot(ann_input, weights_in_1)+err
#        act_2 = T.dot(act_1, weights_1_2)+err
#
#        act_3 = T.dot(act_2, weights_2_2)+err

#        # Build neural-network using tanh activation function

#        act_1 = T.dot(ann_input, weights_in_1)
#        act_2 = T.dot(act_1, weights_1_2)

        act_outs = pm.math.sigmoid(pm.math.dot(act_3s, weights_2_outs)+errs)

        # Binary classification -> Bernoulli likelihood
        out = pm.Normal(
            "out", mu=act_out+act_outs,
            sigma=1, #here i can set sigma = 0.001
            observed=ann_output,
            total_size=Y_train.shape[0],  # IMPORTANT for minibatches
        )
    return neural_network

#https://discourse.pymc.io/t/bayesian-neural-network-no-convergence/3430/8 convergence problem

In [ ]:

# Any results you write to the current directory are saved as output.

# For neural nets with my GPU, RNN doesn't work without this in TF 2.0
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)

print()
print("TF Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
THEANO_FLAGS=blas.ldflags="-L/usr/lib64/ -lblas" ipython


In [ ]:

minibatch_x = pm.Minibatch(X_train, batch_size=512)
minibatch_y = pm.Minibatch(Y_train, batch_size=512)
neural_network_minibatch = construct_nnNEW(minibatch_x, minibatch_y)
with neural_network_minibatch:
    approx = pm.fit(30000, method=pm.ADVI())
    #inference = pm.adamax(learning_rate=0.1)
    #approx = pm.fit(n=30000, method=inference)
    #approx = pm.fit(45000, method=pm.NUTS())
    #trace = pm.sample(3000, tune=1000)


trace = approx.sample(draws=5000)

#pm.summary(trace, varnames=["model_sel"])
#https://discourse.pymc.io/t/bayesian-neural-network-no-convergence/3430/8 convergence problem

In [ ]:
#plt.plot(-inference.hist, label="new ADVI", alpha=0.3)
plt.plot(approx.hist[41000:], label="old ADVI", alpha=0.3)
plt.legend()
plt.ylabel("ELBO")
plt.xlabel("iteration")

plt.plot(-approx.hist[100:], label="new ADVI", alpha=0.3)

In [ ]:
trace = approx.sample(draws=5000)

In [ ]:

pm.summary(trace, var_names="model_sel")



In [ ]:
    pm.set_data(new_data={"ann_input": X_train, "ann_output": Y_train}, model=neural_network_minibatch)
    ppc = pm.sample_posterior_predictive(
        trace, samples=500, progressbar=True, model=neural_network_minibatch, random_seed=132
    )

   
    pred = ppc["out"].mean(axis=0)
    pred.mean(axis=0)
    error = (Y_train-pred)
    plt.plot(pred, Y_train)
    square_error = (Y_train-pred)**2
    error.mean(axis=0)
    square_error.mean(axis=0)
    Y_pred=pred



In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)    
x = range(Y_train.shape[0])
ax1.scatter(x,Y_train, c='r', marker="o", label='observed')
ax1.scatter(x,pred, c='b', marker='x', label='pred')
plt.legend(loc='upper left');
plt.show()


In [ ]:
square_error.mean(axis=0)

In [ ]:
Compute the marginal impact of independent variables

In [ ]:
c= np.ones(X_train.shape[1])  # np.array([1,0,1,1,1])
c[0]=0
Xcounter = X_train*c

#Y = X[:,0]*3-6*X[:,1]*X[:,1]+8*X[:,1]+np.random.normal(0, .2, 1000)+2+latent
Xcounter = Xcounter.astype(floatX)
X_train
Xcounter

In [ ]:
X_train

In [ ]:
pm.set_data(new_data={"ann_input": X_train, "ann_output": Y_train}, model=neural_network_minibatch)
ppc = pm.sample_posterior_predictive(trace, samples=500, progressbar=True, model=neural_network_minibatch, random_seed=132)

In [ ]:
#create counterfactual for training
# I want to capture the impact of X[0]
#c= np.ones(X_train.shape[1])  # np.array([1,0,1,1,1])
#c[0]=0
#Xcounter = X_train*c

#Y = X[:,0]*3-6*X[:,1]*X[:,1]+8*X[:,1]+np.random.normal(0, .2, 1000)+2+latent
#Xcounter = Xcounter.astype(floatX)

pm.set_data(new_data={"ann_input": Xcounter, "ann_output": Y_train}, model=neural_network_minibatch)
ppc_counter = pm.sample_posterior_predictive(
    trace, samples=500, progressbar=True, model=neural_network_minibatch, random_seed=132
)
pred_counter = ppc_counter["out"].mean(axis=0)
pred_counter.mean(axis=0)





In [ ]:
X_train

questo qua sotto da problemi, mi traSFORMA X_TRAIN..IN FEMALE = 1

In [ ]:
Xtreated = X_train*1


In [ ]:
Xtreated.loc[Xtreated["female"] == 0,"female"] = 1


In [ ]:
Xtreated

In [ ]:
X_train

In [ ]:


pm.set_data(new_data={"ann_input": Xtreated, "ann_output": Y_train}, model=neural_network_minibatch)
ppc_treated = pm.sample_posterior_predictive(
    trace, samples=500, progressbar=True, model=neural_network_minibatch, random_seed=132
)
pred_treated = ppc_treated["out"].mean(axis=0)
pred_treated.mean(axis=0)







In [ ]:
impact= ppc_treated['out'] - ppc_counter['out']
Y_impact = impact.mean(axis=0)

In [ ]:
print(Y_impact.mean())
print(Y_impact.std())

In [ ]:


fig = plt.figure()
ax1 = fig.add_subplot(111)    
ax1.scatter(X_train["female"],Y_impact, c='b', marker='x', label='pred')
plt.legend(loc='upper left');
plt.show()


OLS estimates


In [ ]:
#exclude collinear variable


X_ols= X_train
X_ols['const']=1
X_ols= X_ols.loc[:, ~X_ols.columns.isin(['Pf20', 'Pf35', 'Pf35','Pf62','Pf94','Rm12'])]


In [ ]:
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col


reg1 = sm.OLS(endog=Y_train, exog=X_train, missing='drop')
#reg1 = sm.OLS(endog=Y_train, exog=X_ols, missing='drop')
type(reg1)


In [ ]:
results = reg1.fit()
type(results)

In [ ]:
print(results.summary())

In [ ]:
Y_ols = results.predict()

In [ ]:
OLSsquare_error = (Y_train-Y_ols)**2
OLSsquare_error.mean(axis=0)
#square_error.mean(axis=0)


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)    
x = range(Y_train.shape[0])
ax1.scatter(x,Y_train, c='r', marker="o", label='observed')
ax1.scatter(x,Y_ols, c='b', marker='x', label='pred')
plt.legend(loc='upper left');
plt.show()

Nested Bayesian neural network with model selection. This BNN is made up by two different BNN. For each variable the BNN chose which model is more appropiate, applying a specific weight.

In [ ]:
pm.summary(trace, var_names=["model_sel"])